In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select

import mysql.connector
import pandas as pd
from datetime import datetime

In [2]:
# Selenium options
options = webdriver.ChromeOptions()
prefs = {'download.default_directory':'/Home/dev_ws/eda_repo/data',
         'download.prompt_for_download':False}
options.add_experimental_option('prefs', prefs)

# WebDriver
driver = webdriver.Chrome(service=Service("../driver/chromedriver-linux64/chromedriver"),
                          options=options)

# URL
url = "https://portal.kfb.or.kr/fingoods/cofix.php"
driver.get(url)

driver.maximize_window()

# 페이지 스크롤
driver.execute_script("window.scrollTo(0,1100)")

In [6]:
# COFIX 통계 열람 연도 선택
def select_year_option(year_text):
    print(f"selected option: {year_text}년")

    # 드롭다운 요소 찾기
    select_year_element = driver.find_element(By.ID, "BasicYear")
    select_year = Select(select_year_element)
    
    # 연도 선택
    select_year.select_by_visible_text(year_text)
    
    # 보기 버튼 클릭
    view_button = driver.find_element(By.CSS_SELECTOR, ".btn01")
    view_button.click()

    driver.execute_script("window.scrollTo(0,1100)")

# 데이터베이스에 저장
def insert_data_to_cofix(conn, cursor, date, cofix1, cofix2, cofix3):
    try:
        sql_cofix = "INSERT INTO cofix VALUES (%s, %s, %s, %s)"
        values_cofix = (date, cofix1, cofix2, cofix3)
        cursor.execute(sql_cofix, values_cofix)
        conn.commit()
    
    except mysql.connector.Error as error:
        print(f"Error inserting data: {error}")

In [8]:
conn = mysql.connector.connect(
    host = "eda-database.c3oek28e8yta.ap-northeast-2.rds.amazonaws.com",
    port = 3306,
    user = "whskdhs",
    password = "whskdhs1234",
    database = "workspace"
)

cursor = conn.cursor(buffered=True)

select_year_element = driver.find_element(By.ID, "BasicYear")
select_year = Select(select_year_element)
all_options = select_year.options

year_list=[]
for option in all_options:
    year_text = option.text
    year_list.append(year_text)

for year in year_list:
    select_year_option(year)

    # 테이블 찾기
    table = driver.find_element(By.CLASS_NAME, 'resultList_ty02')
    
    # 헤더 추출
    headers = [header.text for header in table.find_elements(By.TAG_NAME, 'th')]
    
    # 행 추출

    for row in table.find_elements(By.TAG_NAME, 'tr')[1:]:  # 헤더 행을 건너뜀.
        cells = row.find_elements(By.TAG_NAME, 'td')
        date_text = cells[0].text
        date = datetime.strptime(date_text, "%Y/%m/%d").date()

        # cofix가 없을 경우 none으로 설정
        cofix1 = float(cells[2].text) if cells[2].text else None
        cofix2 = float(cells[3].text) if cells[3].text else None
        cofix3 = float(cells[4].text) if cells[4].text else None

        print (date, cofix1, cofix2, cofix3)
        insert_data_to_cofix(conn, cursor, date, cofix1, cofix2, cofix3)


selected option: 2024년
2024-09-19 3.36 3.67 3.14
2024-08-16 3.42 3.69 3.15
2024-07-15 3.52 3.73 3.17
2024-06-17 3.56 3.74 3.2
2024-05-16 3.54 3.76 3.17
2024-04-15 3.59 3.78 3.19
2024-03-15 3.62 3.81 3.24
2024-02-15 3.66 3.84 3.29
2024-01-15 3.84 3.87 3.29
selected option: 2023년
2023-12-15 4.0 3.89 3.35
2023-11-15 3.97 3.9 3.33
2023-10-16 3.82 3.88 3.29
2023-09-15 3.66 3.86 3.27
2023-08-16 3.69 3.83 3.21
2023-07-17 3.7 3.8 3.18
2023-06-15 3.56 3.76 3.14
2023-05-15 3.44 3.73 3.09
2023-04-17 3.56 3.71 3.08
2023-03-15 3.53 3.67 3.07
2023-02-15 3.82 3.63 3.02
2023-01-16 4.29 3.52 2.92
selected option: 2022년
2022-12-15 4.34 3.19 2.65
2022-11-15 3.98 2.85 2.36
2022-10-17 3.4 2.52 2.04
2022-09-15 2.96 2.25 1.79
2022-08-16 2.9 2.05 1.62
2022-07-15 2.38 1.83 1.42
2022-06-15 1.98 1.68 1.31
2022-05-16 1.84 1.58 1.22
2022-04-15 1.72 1.5 1.17
2022-03-15 1.7 1.44 1.13
2022-02-15 1.64 1.37 1.08
2022-01-17 1.69 1.3 1.03
selected option: 2021년
2021-12-15 1.55 1.19 0.94
2021-11-15 1.29 1.11 0.89
2021-10-